In [1]:
# -*- coding: utf-8
# VOLT DATA LAB (https://www.voltdata.info/)
# Reinaldo Chaves (@paidatocandeira)
# Detalhamento de como é o gasto com funcionários em cada gabinete e detecta as falhas de transparência do sistema 
#

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import unidecode
import json
import xmltodict
import numpy as np
from datetime import date

Função para retirar acentuação

In [2]:
def f(str):
    return (unidecode.unidecode(str))

Formato do float

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

# Remuneração de servidores da Alesp

Fonte: https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2018-07.html <br>
Último mẽs com informações - julho de 2018

In [4]:
url_chave = ' https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-'
pagamentos = []

Raspa os vencimentos a partir de 2015

In [5]:
for vez in [2015, 2016, 2017, 2018]:
    ano = str(vez)
    for i in range(1, 13):
        if ano == '2018' and i == 8:
            break
        else:
            if i <= 9:
                mes = '0' +  str(i)
            else:
                mes = str(i)
                
            
            link = url_chave + ano + '-' + mes + '.html'
            print(link)
            
            url = urlopen(link)
            
            bsObje = BeautifulSoup(url, "html5lib")
            
            tabela = bsObje.findAll("table", {"class":"tabela_simples"})
            
            conta = 0

            for table_ele in tabela:
                for row in table_ele.findAll('tr'):
                    if conta > 0:
                        cols = row.findAll('td')
                        dicionario = {"ano": ano,
                                      "mes": mes,
                                      "nome_func": cols[0].text.strip(), 
                                      "remuneracao_bruta": cols[1].text.strip(),
                                      "remuneracao_liquida": cols[2].text.strip(),
                                      "tributos": cols[3].text.strip()}
                        pagamentos.append(dicionario)         
                    conta = conta + 1           

 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-01.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-02.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-03.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-04.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-05.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-06.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-07.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-08.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-09.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-10.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-11.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-12.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2016-01.html
 https://www

In [6]:
df_pagamentos = pd.DataFrame(pagamentos)

In [7]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211945 entries, 0 to 211944
Data columns (total 6 columns):
ano                    211945 non-null object
mes                    211945 non-null object
nome_func              211945 non-null object
remuneracao_bruta      211945 non-null object
remuneracao_liquida    211945 non-null object
tributos               211945 non-null object
dtypes: object(6)
memory usage: 9.7+ MB


In [8]:
df_pagamentos.reset_index().head()

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos
0,0,2015,01,ABEL PEREIRA,"13.406,63","10.439,20","2.967,43"
1,1,2015,01,ABENOR GARCIA FERREIRA,"17.507,31","13.968,74","3.538,57"
2,2,2015,01,ABILIO DO LIVRAMENTO P ALVES,"11.160,56","7.919,77","3.240,79"
3,3,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21"
4,4,2015,01,ABRAO BARBOSA DIB,"9.254,85","7.202,63","2.052,22"


In [9]:
df_pagamentos["nome_func"] = df_pagamentos["nome_func"].apply(f)

In [11]:
df_pagamentos.to_excel('resultados/pagamentos_servidores.xlsx',sheet_name='Sheet1')

In [4]:
df_pagamentos = pd.read_excel('resultados/pagamentos_servidores.xlsx',sheet_name='Sheet1')

In [11]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211945 entries, 0 to 211944
Data columns (total 6 columns):
ano                    211945 non-null object
mes                    211945 non-null object
nome_func              211945 non-null object
remuneracao_bruta      211945 non-null object
remuneracao_liquida    211945 non-null object
tributos               211945 non-null object
dtypes: object(6)
memory usage: 9.7+ MB


# Lotação de servidores

Versão com acesso do site que mostra a situação presente

Fonte: https://www.al.sp.gov.br/geral/siga/servidores.jsp?todos=true

In [10]:
url = urlopen("https://www.al.sp.gov.br/geral/siga/servidores.jsp?todos=true")
bsObje =  BeautifulSoup(url, "html5lib")

In [11]:
tabela = bsObje.findAll("table", {"class":"tabela lotacao"})

In [12]:
funcionarios = []
i = 0
for table_ele in tabela:
    for row in table_ele.findAll('tr'):
        if i > 1:
            cols = row.findAll('td')
            dicionario = {"nome_func": cols[0].text.strip(), 
                              "tipo_cargo1": cols[1].text.strip(),
                              "tipo_cargo2": cols[2].text.strip(),
                              "lotacao": cols[3].text.strip()}
            funcionarios.append(dicionario)         
        i = i + 1

In [13]:
df_lotacao_presente = pd.DataFrame(funcionarios)

In [14]:
df_lotacao_presente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27946 entries, 0 to 27945
Data columns (total 4 columns):
lotacao        27946 non-null object
nome_func      27946 non-null object
tipo_cargo1    27946 non-null object
tipo_cargo2    27946 non-null object
dtypes: object(4)
memory usage: 873.4+ KB


In [16]:
df_lotacao_presente.reset_index().head()

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,0,GABINETE DEP. ENIO TATTO,ABALDINO FERREIRA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
1,1,Ex-funcionário,ABDALLAH ABRÃO AUAD,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
2,2,Ex-funcionário,ABDIAS CRISPIM DE ALMEIDA,,
3,3,Ex-funcionário,ABDIAS DA SILVA,AUXILIAR PARLAMENTAR\n\t\t\t\t\t\t\t\n\t\t\t\t...,COMISSÃO
4,4,Ex-funcionário,ABDULNASSER MOHAMAD ABOU NASSIF,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO


In [15]:
df_lotacao_presente["nome_func"] = df_lotacao_presente["nome_func"].apply(f)

In [18]:
df_lotacao_presente.reset_index().head(10)

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,0,GABINETE DEP. ENIO TATTO,ABALDINO FERREIRA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
1,1,Ex-funcionário,ABDALLAH ABRAO AUAD,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
2,2,Ex-funcionário,ABDIAS CRISPIM DE ALMEIDA,,
3,3,Ex-funcionário,ABDIAS DA SILVA,AUXILIAR PARLAMENTAR\n\t\t\t\t\t\t\t\n\t\t\t\t...,COMISSÃO
4,4,Ex-funcionário,ABDULNASSER MOHAMAD ABOU NASSIF,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
5,5,Ex-funcionário,ABEDIAS PEREIRA DE ARAUJO,,
6,6,GABINETE DEP. CAUÊ MACRIS,ABEL CASTILHO,COMISSIONADOS,COMISSIONAMENTO SEM PREJUÍZO
7,7,Ex-funcionário,ABEL GARCIA DE SOUZA,,
8,8,Ex-funcionário,ABEL HILARION FERNANDEZ JUNIOR,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
9,9,Ex-funcionário,ABEL PEREIRA,,


Seleciona apenas nos gabinetes

In [16]:
str_choice = "GABINETE" 
gabinetes_presente = df_lotacao_presente[df_lotacao_presente['lotacao'].\
                       str.contains(str_choice, na=False)]

In [44]:
gabinetes_presente.to_excel('resultados/lotacao_servidores_atual_gabinetes.xlsx',sheet_name='Sheet1')

### Versão que acessa o histórico de lotações

Fonte: https://www.al.sp.gov.br/dados-abertos/recurso/181

Transforma arquivo XML em JSON 

In [34]:
with open("dados/administracao/funcionarios_lotacoes.xml", 'r') as f:
    xmlString = f.read()
jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
with open("output.json", 'w') as f:
    f.write(jsonString)

In [35]:
arquivo_json = open('output.json', 'r')

In [36]:
dados_json = json.load(arquivo_json)

In [37]:
#dados_json

Transforma JSON em dataframe

In [38]:
lista = dados_json['HistoricoFuncionariosLotacoes']['LotacaoFuncionario']

In [39]:
df_lotacoes = pd.DataFrame(lista, columns = ['DataFim', 'DataInicio', 'IdUA', 'NomeFuncionario', 'NomeUA'])

In [40]:
df_lotacoes['DataFim'] = pd.to_datetime(df_lotacoes['DataFim'])
df_lotacoes['DataInicio'] = pd.to_datetime(df_lotacoes['DataInicio'])

In [43]:
df_lotacoes["NomeFuncionario"] = df_lotacoes["NomeFuncionario"].apply(f)

In [44]:
df_lotacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51648 entries, 0 to 51647
Data columns (total 5 columns):
DataFim            47496 non-null datetime64[ns]
DataInicio         51648 non-null datetime64[ns]
IdUA               51648 non-null object
NomeFuncionario    51648 non-null object
NomeUA             51648 non-null object
dtypes: datetime64[ns](2), object(3)
memory usage: 2.0+ MB


In [45]:
df_lotacoes.reset_index().head()

,index,DataFim,DataInicio,IdUA,NomeFuncionario,NomeUA
0,0,NaT,2018-08-10 03:00:00,20440,ABALDINO FERREIRA DA SILVA,Gabinete do Deputado ENIO TATTO
1,1,2008-08-28 03:00:00,2007-03-27 03:00:00,20447,ABDALLAH ABRAO AUAD,Gabinete do Deputado FERNANDO CAPEZ
2,2,2002-08-08 03:00:00,2001-04-06 03:00:00,53487,ABDIAS DA SILVA,SERVIÇO DE CONTROLE DE FROTA
3,3,2007-03-14 03:00:00,2006-11-08 02:00:00,20385,ABDIAS DA SILVA,Gabinete do Deputado ROMEU TUMA
4,4,2005-05-06 03:00:00,2004-06-30 03:00:00,53487,ABDULNASSER MOHAMAD ABOU NASSIF,SERVIÇO DE CONTROLE DE FROTA


In [26]:
df_lotacoes.to_excel('resultados/lotacao_servidores_historico.xlsx',sheet_name='Sheet1')

In [ ]:
df_lotacoes = pd.read_excel('resultados/lotacao_servidores_historico.xlsx',sheet_name='Sheet1')

In [27]:
df_lotacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51648 entries, 0 to 51647
Data columns (total 5 columns):
DataFim            47496 non-null datetime64[ns]
DataInicio         51648 non-null datetime64[ns]
IdUA               51648 non-null object
NomeFuncionario    51648 non-null object
NomeUA             51648 non-null object
dtypes: datetime64[ns](2), object(3)
memory usage: 2.0+ MB


IdUA é o código da unidade administrativa

# Aqui um problema: há funcionários homônimos<br>
# A Alesp não divulga um código identificador único dos servidores<br>
# Aqui o exemplo de possíveis homônimos na base de lotações

In [42]:
df_lotacoes['NomeFuncionario'].value_counts()

PAULO ROBERTO DE MORAIS ALMEIDA           120
MARCIO GAZZO                               71
SERGIO SICILIANO                           65
REGINA DE SOUZA MOREIRA                    55
JOAO BOSCO DA SILVA                        53
SHIGUEO YAMAMOTO                           53
SIDNEY GONCALVES IZZO                      52
MARCOS ANTONIO DEMATEI PIETRAFESA          49
MARIA TERESA DE OLIVEIRA ROSA              49
MARCIO LUIZ DE CAMPOS MARQUES              49
MARTA JANETE GARCIA DE SOUZA               47
ISABEL CRISTINA ROTHER BUHLER DE GODOY     47
RITA DE CASSIA CESAR GOES                  47
PATRICIA ROSSET                            46
CARLOS EDUARDO GOMES                       45
PAULO SERGIO CHAMMAS                       44
GISELDA FARIAS DOS SANTOS                  44
CELSO LEMOS                                44
CARLOS CESAR CARREGARI                     43
SUZANA KENIGER LISBOA                      43
RITA DE CASSIA ATALLA SIMONATO             43
ZENILDA DOS SANTOS BIZERRA        

Número de funcionários por lotação

In [43]:
df_lotacoes['NomeUA'].value_counts()

PRESIDÊNCIA                                            2038
2ª SECRETARIA                                          1398
1ª SECRETARIA                                          1246
Gabinete do Deputado                                   1137
LIDERANÇA DO PSDB                                      1083
LIDERANÇA DO PT                                         992
DEPARTAMENTO ADMINISTRATIVO                             842
DIVISÃO DE TRANSPORTES                                  794
LIDERANÇA DO PTB                                        695
LIDERANÇA DO PMDB                                       680
SERVIÇO DE CONTROLE DE FROTA                            653
AFASTADOS                                               600
DIRETORIA GERAL                                         597
ASSISTÊNCIA POLICIAL MILITAR                            531
2ª VICE PRESIDÊNCIA                                     456
3ª SECRETARIA                                           441
LIDERANÇA DO PFL                        

Seleciona apenas Gabinete

In [48]:
str_choice = "Gabinete" 
gabinetes = df_lotacoes[df_lotacoes['NomeUA'].\
                       str.contains(str_choice, na=False)]

In [49]:
gabinetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23283 entries, 0 to 51647
Data columns (total 5 columns):
DataFim            20976 non-null datetime64[ns]
DataInicio         23283 non-null datetime64[ns]
IdUA               23283 non-null object
NomeFuncionario    23283 non-null object
NomeUA             23283 non-null object
dtypes: datetime64[ns](2), object(3)
memory usage: 1.1+ MB


In [51]:
#gabinetes.reset_index()

Funcionários por gabinete - historicamente

In [46]:
gabinetes['NomeUA'].value_counts()

Gabinete do Deputado                                1137
Gabinete do Deputado EDMIR CHEDID                    309
Gabinete do Deputado ENIO TATTO                      302
Gabinete do Deputado BARROS MUNHOZ                   207
Gabinete do Deputado JOÃO CARAMEZ                    204
Gabinete do Deputado JOSÉ ZICO PRADO                 202
Gabinete do Deputado LUIZ CARLOS GONDIM              191
Gabinete do Deputado ROQUE BARBIERE                  188
Gabinete do Deputado VITOR SAPIENZA                  187
Gabinete do Deputado Orlando Morando                 184
Gabinete do Deputado GERALDO VINHOLI                 181
Gabinete do Deputado RAFAEL SILVA                    180
Gabinete do Deputado MILTON VIEIRA                   173
Gabinete do Deputado CELSO GIGLIO                    171
Gabinete do Deputado Vanessa Damo                    167
Gabinete do Deputado BETH SAHÃO                      165
Gabinete do Deputado MARIA LÚCIA AMARY               165
Gabinete do Deputado ALDO DEMAR

In [49]:
mask = (gabinetes['DataInicio'].dt.year >= 2015) | (gabinetes['DataFim'].isnull())

gabinetes = gabinetes[mask]

Funcionários por gabinete - última legislatura

In [50]:
gabinetes['NomeUA'].value_counts()

Gabinete do Deputado GIL LANCASTER                        124
Gabinete do Deputado WELLINGTON MOURA                     115
Gabinete do Deputado ENIO TATTO                           113
Gabinete do Deputado PROFESSOR AURIEL                      89
Gabinete do Deputado BARROS MUNHOZ                         87
Gabinete do Deputado EDMIR CHEDID                          87
Gabinete do Deputado FERNANDO CURY                         86
Gabinete do Deputado CELSO NASCIMENTO                      86
Gabinete do Deputado JORGE WILSON XERIFE DO CONSUMIDOR     84
Gabinete do Deputado CARLÃO PIGNATARI                      84
Gabinete do Deputado TEONILIO BARBA                        83
Gabinete do Deputado CEZINHA DE MADUREIRA                  81
Gabinete do Deputado JOSÉ ZICO PRADO                       79
Gabinete do Deputado CLÉLIA GOMES                          78
Gabinete do Deputado MÁRCIO CAMARGO                        77
Gabinete do Deputado RICARDO MADALENA                      75
Gabinete

In [51]:
gabinetes.reset_index().head(10)

,index,DataFim,DataInicio,IdUA,NomeFuncionario,NomeUA
0,0,NaT,2018-08-10 03:00:00,20440,ABALDINO FERREIRA DA SILVA,Gabinete do Deputado ENIO TATTO
1,5,NaT,2018-07-04 03:00:00,20373,ABEL CASTILHO,Gabinete do Deputado CAUÊ MACRIS
2,32,NaT,2018-03-14 03:00:00,20413,ABNER REISES BARBOSA,Gabinete do Deputado CELSO NASCIMENTO
3,35,NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
4,36,2017-05-08 03:00:00,2016-06-01 03:00:00,20454,ABRAHAO RICARDO HACKME,Gabinete do Deputado SEBASTIÃO SANTOS
5,37,2016-04-12 03:00:00,2015-06-23 03:00:00,20454,ABRAHAO RICARDO HACKME,Gabinete do Deputado SEBASTIÃO SANTOS
6,38,2016-05-31 03:00:00,2016-05-11 03:00:00,20454,ABRAHAO RICARDO HACKME,Gabinete do Deputado SEBASTIÃO SANTOS
7,39,NaT,2017-05-09 03:00:00,20454,ABRAHAO RICARDO HACKME,Gabinete do Deputado SEBASTIÃO SANTOS
8,40,NaT,2018-04-03 03:00:00,20452,ABRAAO MOREIRA DA SILVA,Gabinete do Deputado RODRIGO MORAES
9,48,NaT,2015-03-26 03:00:00,20435,ACLERISTON PEREIRA MARTINS,Gabinete do Deputado CORONEL CAMILO


In [52]:
len(gabinetes['NomeFuncionario'].value_counts())

4206

In [29]:
gabinetes.to_excel('resultados/funcionarios_gabinetes.xlsx',sheet_name='Sheet1')

# Aqui o exemplo de possíveis homônimos na base de pagamentos

In [47]:
df_pagamentos['nome_func'].value_counts()

JOSE CARLOS PEREIRA                           126
JOSE ROBERTO DA SILVA                         109
SHIGUERO ORITA                                 86
LUIZ FERNANDO DE CARVALHO                      86
JOSE RUIZ NETTO                                86
ANTONIO DOS SANTOS                             86
REINALDO DA SILVA                              86
RICARDO DE OLIVEIRA                            86
JOAO CARLOS FRANCISCO DA SILVA                 86
ANA MARIA DA SILVA                             86
SONIA MARIA CARNEIRO D ALBUQUERQUE             86
CLAUDIO FERREIRA                               86
JOSE GONCALVES                                 86
JOSE CARLOS DUARTE DE TOLEDO                   84
SUELI SOARES FARIA DE OLIVEIRA                 82
WILSON JOSE RIBEIRO                            69
VICTORINA THEREZA FRUGOLI                      67
MARIA RODRIGUES FERREIRA DE ANDRADE            64
MARIA APARECIDA DE SOUZA                       60
PAULO EDUARDO ALVES FERREIRA                   60


In [31]:
#df_pagamentos = df_pagamentos.drop_duplicates(['nome_func'])

# Cruza remuneração e lotação - APENAS PARA TESTAR - SERÁ NECESSÁRIO FILTRAR HOMÔNIMOS

In [53]:
servidores_gabinetes = pd.merge(df_pagamentos, gabinetes, left_on='nome_func', right_on='NomeFuncionario')

In [54]:
servidores_gabinetes.reset_index().head()

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,DataFim,DataInicio,IdUA,NomeFuncionario,NomeUA
0,0,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
1,1,2015,02,ABNEIA BUFFOLIN BROCANELLI,"1.533,16","1.370,40","162,76",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
2,2,2015,03,ABNEIA BUFFOLIN BROCANELLI,"0,00","0,00","0,00",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
3,3,2015,04,ABNEIA BUFFOLIN BROCANELLI,"0,00","-7,43","7,43",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
4,4,2015,05,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.348,13","992,44",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO


# Verifica nomes não encontrados na lista de remunerações  <br>
# Ou seja, são servidores que não têm ou não tiveram no passado o salário divulgado

In [59]:
nomes_naoencontrados = pd.merge(df_pagamentos,
                        gabinetes,
                        left_on='nome_func', 
                        right_on='NomeFuncionario',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [60]:
rdf['NomeFuncionario'].value_counts()

THAUZER GUTIERREZ RODRIGUES MARTINS PINTO DOS SANTOS       3
MARTA JANETE GARCIA DE SOUZA                               3
ANGELITA PEREIRA DA CRUZ                                   2
GILMAR ANDRE DOS SANTOS NEVES DE LAVOR                     2
ADOLPHO DOS SANTOS GAMBOA JUNIOR                           2
FRANK MARQUES JUNIOR                                       2
PRISCILA VERISSIMO LEITE DE MORAES                         2
MARIA DE FATIMA PEREIRA DE OLIVEIRA ARAUJO                 2
EDISON JOSE UTINETTI                                       2
CARMEN SILVIA VALIO DE ARAUJO MARTINS                      2
SONIA MARIA OLIVEIRA DO CARMO                              2
CARLA SOUSA VASCONCELOS DE ALMEIDA                         2
SILVIA MARIA KURY DE SOUZA                                 2
ARLINDO SOUZA MELO NETO                                    2
JOSE CARLOS DUARTE TOLEDO                                  1
ANA CLAUDIA GALVAO SILVA                                   1
JACQUES SOARES LEITE SIQ

In [61]:
rdf['NomeUA'].value_counts()

Gabinete do Deputado ROQUE BARBIERE               11
Gabinete do Deputado GIL LANCASTER                 9
Gabinete do Deputado GILMAR GIMENES                5
Gabinete do Deputado GUSTAVO PETTA                 5
Gabinete do Deputado MARTA COSTA                   4
Gabinete do Deputado DELEGADO OLIM                 4
Gabinete do Deputado FERNANDO CURY                 4
Gabinete do Deputado LÉO OLIVEIRA                  4
Gabinete do Deputado MARIA LÚCIA AMARY             4
Gabinete do Deputado FERNANDO CAPEZ                3
Gabinete do Deputado LUIZ FERNANDO T. FERREIRA     3
Gabinete do Deputado CLÉLIA GOMES                  3
Gabinete do Deputado ROGÉRIO NOGUEIRA              3
Gabinete do Deputado BARROS MUNHOZ                 3
Gabinete do Deputado EDMIR CHEDID                  3
Gabinete do Deputado ANDRÉ SOARES                  3
Gabinete do Deputado CAMPOS MACHADO                3
Gabinete do Deputado RAUL MARCELO                  3
Gabinete do Deputado MÁRCIO CAMARGO           

In [38]:
rdf.to_excel('resultados/servidores_naoencontrados.xlsx',sheet_name='Sheet1')

# Verifica nomes não encontrados na lista de remunerações para os funcionários de gabinete do presente - apenas 1 deles não é comissionado <br>
# Ou seja, são servidores que não têm o salário divulgado

In [71]:
nomes_naoencontrados = pd.merge(df_pagamentos,
                        gabinetes_presente,
                        left_on='nome_func', 
                        right_on='nome_func',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [72]:
rdf['nome_func'].value_counts()

EDIMARA CRISTINA DE MIRANDA ANDREUCCI NAJAR HERNANDEZ    1
CLAUDIO ARAUJO MACHADO                                   1
TARCISIO DANIEL BOAVENTURA                               1
LUIZ CARLOS DE ALMEIDA PRADO ALVES FILHO                 1
JURANDIR BENEDITO SILVA                                  1
JACQUES SOARES LEITE SIQUEIRA                            1
ANA CLAUDIA GALVAO SILVA                                 1
DANILO GOMES DE SOUZA                                    1
RAUL MARCEL DA SILVA                                     1
ANTONIO HENRIQUE ASSIS BRAZAO DE SOUZA                   1
ANDRE ABEL DA SILVA                                      1
VICTOR RAMALHOSO GUERREIRO                               1
AYL DE ANDRADE MARQUES FILHO                             1
LILIANE MOREIRA DE OLIVEIRA LIRA                         1
SHEILLA ISAIAS PEREIRA DO NASCIMENTO                     1
JAILTON REIS DOS SANTOS                                  1
MARTA JANETE GARCIA DE SOUZA                            

In [73]:
rdf['tipo_cargo2'].value_counts()

COMISSÃO                                                                    95
COMISSIONAMENTO SEM PREJUÍZO                                                 4
COMISSÃO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ EFETIVO                          1
EFETIVO                                                                      1
COMISSIONAMENTO SEM PREJUÍZO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ COMISSÃO     1
Name: tipo_cargo2, dtype: int64

In [74]:
nome = rdf[(rdf['tipo_cargo2'] == 'EFETIVO')]
nome

,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
212243,NaN,NaN,LUIZ CARLOS DE ALMEIDA PRADO ALVES FILHO,NaN,NaN,NaN,GABINETE DEP. FERNANDO CURY,TÉCNICO LEGISLATIVO\n\t\t\t\t\t\t\t\n\t\t\t\t\...,EFETIVO


In [75]:
servidores_gabinetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133970 entries, 0 to 133969
Data columns (total 11 columns):
ano                    133970 non-null object
mes                    133970 non-null object
nome_func              133970 non-null object
remuneracao_bruta      133970 non-null object
remuneracao_liquida    133970 non-null object
tributos               133970 non-null object
DataFim                68667 non-null datetime64[ns]
DataInicio             133970 non-null datetime64[ns]
IdUA                   133970 non-null object
NomeFuncionario        133970 non-null object
NomeUA                 133970 non-null object
dtypes: datetime64[ns](2), object(9)
memory usage: 12.3+ MB


In [76]:
servidores_gabinetes.reset_index().head(10)

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,DataFim,DataInicio,IdUA,NomeFuncionario,NomeUA
0,0,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
1,1,2015,02,ABNEIA BUFFOLIN BROCANELLI,"1.533,16","1.370,40","162,76",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
2,2,2015,03,ABNEIA BUFFOLIN BROCANELLI,"0,00","0,00","0,00",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
3,3,2015,04,ABNEIA BUFFOLIN BROCANELLI,"0,00","-7,43","7,43",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
4,4,2015,05,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.348,13","992,44",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
5,5,2015,06,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
6,6,2015,07,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
7,7,2015,08,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
8,8,2015,09,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO
9,9,2015,10,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",NaT,2015-04-28 03:00:00,20413,ABNEIA BUFFOLIN BROCANELLI,Gabinete do Deputado CELSO NASCIMENTO


# Transforma valores para padrão decimal que permite operações matemáticas

In [17]:
def limpeza1(linha):
    valor = linha["remuneracao_bruta"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

def limpeza2(linha):
    valor = linha["remuneracao_liquida"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

In [18]:
df_pagamentos["remuneracao_bruta"] = df_pagamentos.apply(limpeza1, axis=1)
df_pagamentos["remuneracao_liquida"] = df_pagamentos.apply(limpeza2, axis=1)

df_pagamentos['remuneracao_bruta'] = df_pagamentos.remuneracao_bruta.astype('float64')
df_pagamentos['remuneracao_liquida'] = df_pagamentos.remuneracao_liquida.astype('float64')

In [19]:
df_pagamentos.reset_index().head(10)

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos
0,0,2015,01,ABEL PEREIRA,"13,406.63","10,439.20","2.967,43"
1,1,2015,01,ABENOR GARCIA FERREIRA,"17,507.31","13,968.74","3.538,57"
2,2,2015,01,ABILIO DO LIVRAMENTO P ALVES,"11,160.56","7,919.77","3.240,79"
3,3,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4,599.47","3,758.26","841,21"
4,4,2015,01,ABRAO BARBOSA DIB,"9,254.85","7,202.63","2.052,22"
5,5,2015,01,ACACIA DE ASSIS NEGREIROS,"6,658.91","5,961.02","697,89"
6,6,2015,01,ACACIO GONSALES FERREIRA,"2,858.20","2,451.84","406,36"
7,7,2015,01,ACIDALIA ARGENTE,"11,288.34","8,883.67","2.404,67"
8,8,2015,01,ACY DUARTE CINQUE,"11,849.48","8,904.21","2.945,27"
9,9,2015,01,ADALBERTO CRISTOVAO MOREIRA,"16,199.44","11,155.76","5.043,68"


# Análise de gastos de apenas um mês - julho 2018
# A análise de cada mês em separado - retirando homônimos é a forma mais precisa de trabalhar porque falta mais transparência da Alesp

In [20]:
julho_pagamentos = df_pagamentos[(df_pagamentos['ano'] == '2018') &
                            (df_pagamentos['mes'] == '07') 
                           ]

In [21]:
julho_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996 entries, 206949 to 211944
Data columns (total 6 columns):
ano                    4996 non-null object
mes                    4996 non-null object
nome_func              4996 non-null object
remuneracao_bruta      4996 non-null float64
remuneracao_liquida    4996 non-null float64
tributos               4996 non-null object
dtypes: float64(2), object(4)
memory usage: 273.2+ KB


Total nos gabinetes em julho

In [22]:
gabinetes_presente['nome_func'].nunique()

2349

Comissionados

In [23]:
#str_choice = "COMISSÃO" 
#comissionados = gabinetes_presente[gabinetes_presente['tipo_cargo2'].\
#                       str.contains(str_choice, na=False)]
comissionados = gabinetes_presente[(gabinetes_presente['tipo_cargo2'] == 'COMISSÃO')]
comissionados['nome_func'].nunique()

2286

Homônimos que não é possível diferenciar

In [24]:
julho_pagamentos['nome_func'].value_counts()

JOSE CARLOS PEREIRA                         3
JOSE ROBERTO DA SILVA                       2
CLAUDIO FERREIRA                            2
ANTONIO DOS SANTOS                          2
RICARDO DE OLIVEIRA                         2
CARLOS ROBERTO SPARTANI PIRANI              2
MARIA APARECIDA DE SOUZA                    2
SHIGUERO ORITA                              2
ANA MARIA DA SILVA                          2
JOSE RUIZ NETTO                             2
MARILUCIA PARDINI OLBRICH ZANELATO BUCHI    2
JOSE CARLOS DUARTE DE TOLEDO                2
LUIZ FERNANDO DE CARVALHO                   2
SUELI SOARES FARIA DE OLIVEIRA              2
JOSE GONCALVES                              2
PAULO EDUARDO ALVES FERREIRA                2
JOAO CARLOS FRANCISCO DA SILVA              2
REINALDO DA SILVA                           2
SONIA MARIA CARNEIRO D ALBUQUERQUE          2
SANDRA FILOMENA L DOS SANTOS                1
VINICIUS DE MELO CARDOSO                    1
EDI CARLOS VIEIRA                 

Retira duplicados

In [25]:
julho_pagamentos = julho_pagamentos.drop_duplicates(['nome_func'])

In [26]:
gabinetes_presente = gabinetes_presente.drop_duplicates(['nome_func'])

Cruza com os gabinetes - tira nomes duplicados também em gabinetes

In [27]:
servidores_gabinetes_julho = pd.merge(julho_pagamentos, gabinetes_presente, left_on='nome_func', right_on='nome_func')

In [28]:
servidores_gabinetes_julho.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2236 entries, 0 to 2235
Data columns (total 9 columns):
ano                    2236 non-null object
mes                    2236 non-null object
nome_func              2236 non-null object
remuneracao_bruta      2236 non-null float64
remuneracao_liquida    2236 non-null float64
tributos               2236 non-null object
lotacao                2236 non-null object
tipo_cargo1            2236 non-null object
tipo_cargo2            2236 non-null object
dtypes: float64(2), object(7)
memory usage: 174.7+ KB


In [29]:
nomes_naoencontrados = pd.merge(julho_pagamentos,
                        gabinetes_presente.drop_duplicates('nome_func'),
                        left_on='nome_func', 
                        right_on='nome_func',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

rdf['tipo_cargo2'].value_counts()

COMISSÃO                                                                    104
COMISSIONAMENTO SEM PREJUÍZO                                                  6
COMISSIONAMENTO SEM PREJUÍZO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ COMISSÃO      1
EFETIVO                                                                       1
COMISSÃO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ EFETIVO                           1
Name: tipo_cargo2, dtype: int64

In [30]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 4976 to 5088
Data columns (total 9 columns):
ano                    0 non-null object
mes                    0 non-null object
nome_func              113 non-null object
remuneracao_bruta      0 non-null float64
remuneracao_liquida    0 non-null float64
tributos               0 non-null object
lotacao                113 non-null object
tipo_cargo1            113 non-null object
tipo_cargo2            113 non-null object
dtypes: float64(2), object(7)
memory usage: 8.8+ KB


In [63]:
rdf.to_excel('resultados/servidores_semsalariodivulgado.xlsx',sheet_name='Sheet1')

Gabinetes com gastos

In [45]:
#a = len(servidores_gabinetes_julho['lotacao'].value_counts())
#a
servidores_gabinetes_julho['lotacao'].nunique()

101

Soma dos gastos com servidores

In [46]:
"${:,.2f}".format(servidores_gabinetes_julho.remuneracao_bruta.sum())
#servidores_gabinetes_julho['remuneracao_bruta'].sum()

'$18,825,139.33'

Média

In [47]:
servidores_gabinetes_julho['remuneracao_bruta'].mean()

8419.114190518783

Mediana

In [34]:
"${:,.0f}".format(servidores_gabinetes_julho.remuneracao_bruta.median())

'$6,530'

Gastos com servidores por gabinete - sem contar homônimos

In [35]:
servidores_gabinetes_julho.groupby("lotacao").remuneracao_liquida.sum().reset_index().sort_values("remuneracao_liquida", ascending=False)

,lotacao,remuneracao_liquida
8,GABINETE DEP. ALDO DEMARCHI,"223,293.18"
43,GABINETE DEP. FERNANDO CAPEZ,"200,979.43"
10,GABINETE DEP. ANA DO CARMO,"198,855.39"
62,GABINETE DEP. LUIZ TURCO,"194,680.31"
90,GABINETE DEP. ROGÉRIO NOGUEIRA,"183,070.67"
38,GABINETE DEP. EDMIR CHEDID,"182,801.44"
89,GABINETE DEP. RODRIGO MORAES,"179,932.84"
60,GABINETE DEP. LUIZ CARLOS GONDIM,"176,779.53"
93,GABINETE DEP. TEONILIO BARBA,"174,406.65"
41,GABINETE DEP. ESTEVAM GALVÃO,"173,397.34"


In [48]:
servidores_gabinetes_julho.to_excel('resultados/gastos_servidores_julho.xlsx',sheet_name='Sheet1')

In [49]:
gasto_total = servidores_gabinetes_julho.groupby("lotacao").remuneracao_liquida.sum().reset_index().sort_values("remuneracao_liquida", ascending=False)

In [50]:
gasto_total['remuneracao_liquida'].mean()

142514.90237623762

Porcentagens em relação ao total - Gastos com servidores por gabinete - total mensal - sem contar homônimos

In [51]:
# Agrupamento dos gastos por categoria
df_gabinete = servidores_gabinetes_julho.groupby(['lotacao']).agg({'remuneracao_bruta': sum})\
                                .sort_values('remuneracao_bruta', ascending = False)\
                                .reset_index().copy()

# Cálculo de porcentagem de cada categoria no total de gastos
df_gabinete['porcentagem'] = (df_gabinete['remuneracao_bruta'] / df_gabinete['remuneracao_bruta'].sum()) * 100

# Formatação de números para duas casas decimais
df_gabinete['remuneracao_bruta']['remuneracao_bruta'] = df_gabinete['remuneracao_bruta'].map('{:,.2f}'.format)
df_gabinete['porcentagem'] = df_gabinete['porcentagem'].map('{:,.2f}'.format)

# Resultado
df_gabinete.reset_index().head(10)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,lotacao,remuneracao_bruta,porcentagem
0,0,GABINETE DEP. ALDO DEMARCHI,"296,176.65",1.57
1,1,GABINETE DEP. FERNANDO CAPEZ,"267,229.72",1.42
2,2,GABINETE DEP. ANA DO CARMO,"258,767.89",1.37
3,3,GABINETE DEP. LUIZ TURCO,"253,377.06",1.35
4,4,GABINETE DEP. ROGÉRIO NOGUEIRA,"247,269.38",1.31
5,5,GABINETE DEP. EDMIR CHEDID,"240,248.31",1.28
6,6,GABINETE DEP. RODRIGO MORAES,"236,039.87",1.25
7,7,GABINETE DEP. LUIZ CARLOS GONDIM,"233,583.00",1.24
8,8,GABINETE DEP. AFONSO LOBATO,"226,592.05",1.20
9,9,GABINETE DEP. ESTEVAM GALVÃO,"226,318.11",1.20


In [52]:
df_gabinete.to_excel('resultados/maiores_gastos_comservidores_gabinetes.xlsx',sheet_name='Sheet1')

Quantidade de servidores por gabinete - sem contar homônimos

In [41]:
servidores_gabinetes_julho.groupby("lotacao").nome_func.count().sort_values(ascending=False)

lotacao
GABINETE DEP. LUIZ TURCO                   33
GABINETE DEP. ANA DO CARMO                 32
GABINETE DEP. TEONILIO BARBA               31
GABINETE DEP. PROFESSOR AURIEL             30
GABINETE DEP. GILENO GOMES                 30
GABINETE DEP. ENIO TATTO                   30
GABINETE DEP. LUIZ FERNANDO T. FERREIRA    29
GABINETE DEP. ALENCAR SANTANA BRAGA        29
GABINETE DEP. ANALICE FERNANDES            29
GABINETE DEP. GERALDO CRUZ                 29
GABINETE DEP. CORONEL CAMILO               29
GABINETE DEP. CARLOS NEDER                 29
GABINETE DEP. JORGE CARUSO                 28
GABINETE DEP. CAUÊ MACRIS                  28
GABINETE DEP. MARCOS DAMASIO               28
GABINETE 48                                28
GABINETE DEP. JOSÉ ZICO PRADO              27
GABINETE DEP. LUIZ CARLOS GONDIM           27
GABINETE DEP. CAIO FRANÇA                  27
GABINETE DEP. REINALDO ALGUZ               27
GABINETE DEP. RODRIGO MORAES               27
GABINETE DEP. ADILSON ROSS

Servidores com maiores remunerações

In [42]:
servidores_gabinetes_julho.groupby("nome_func").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False).head(10)

,nome_func,remuneracao_bruta
1467,MARIA APARECIDA JANUARIO,"25,322.25"
198,ANTONIO CARLOS DIAS PEREIRA FILHO,"25,322.25"
1468,MARIA BETANIA DE AQUINO MORAES,"25,322.25"
1511,MARIA TERESA DOS SANTOS SIQUEIRA,"25,322.25"
1293,LUIS EDUARDO LACERDA,"25,322.25"
154,ANDRE LUIS CHUFULI AMOROSO,"25,322.25"
1120,JOSE LUIZ ALVES,"25,322.25"
1092,JOSE CAVALLI JUNIOR,"25,322.25"
1382,MARCELO SOUZA SERPA,"25,322.25"
695,EVALDO PEREIRA DA SILVA,"25,322.25"


Mostra o gabinete relacionado de um servidor

In [43]:
nome = servidores_gabinetes_julho[(servidores_gabinetes_julho['nome_func'] == 'LUIS EDUARDO LACERDA')].copy()
nome

,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
1293,2018,07,LUIS EDUARDO LACERDA,"25,322.25","18,559.57","6.762,68",GABINETE DEP. ROBERTO MASSAFERA,ASSISTENTE TÉCNICO PARLAMENTAR\n\t\t\t\t\t\t\t...,COMISSÃO
